In [1]:
### Packages .........................................................
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from scipy import ndimage as ndi
from scipy.spatial import ConvexHull
from skimage import feature,io, data, morphology, measure, filters, restoration
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.measure import label, regionprops
from skimage.morphology import binary_opening, binary_closing, binary_dilation, binary_erosion
from skimage import img_as_float, img_as_uint, img_as_ubyte
from skimage.filters import median, gaussian
from skimage.morphology import disk, ball
from skimage import exposure
import time
import pathlib
import pandas as pd

In [2]:
### Lecture de l'image .........................................................
# Emplacement des fichiers
dir_data = pathlib.Path('Test3_1751_1950')
path = dir_data

In [ ]:
# Read image sequence
print("Reading image ..........")

i_0=0 #1rst image to read
i_1=199 #last image to read

#Shape of an image
img_2 = io.imread(str(path / '%s%05d.tif') %("Images_L_500x500x200_",i_0))
a1, b1= img_2.shape
print('Image shape: %d, %d' %(a1,b1))

#Initialisation
Images=np.zeros((i_1-i_0+1,a1,b1))
Med_3=np.zeros((i_1-i_0+1,a1,b1))

#Read images
for i in range (i_0, i_1+1):    
    img= io.imread(str(path / '%s%05d.tif') %("Images_L_500x500x200_", i)) 
    Images[i-i_0,:,:]= img
    med = median(img, disk(3)) #median filter (2D)
    Med_3[i-i_0,:,:]= med
print(path,"read")

# Gaussian filter (3D)
Gau_3 = gaussian(Images,0.8,preserve_range=True)

# Saving images
np.save('Test3_1751_1950\Images.npy',Images)
np.save('Test3_1751_1950\Images_Median.npy',Med_3)
np.save('Test3_1751_1950\Images_Gaussian.npy',Gau_3)

Reading image ..........
Image shape: 500, 500


In [3]:
# Relaod images
Images=np.load('Test3_1751_1950\Images.npy')
Med_3 = np.load('Test3_1751_1950\Images_Median.npy')

In [4]:
#Example of Original & filter image
i_x = 100
#Show images & their histograms
fig,a2=  plt.subplots(2,2, figsize=(10, 10))

a2[0][0].imshow(Images[i_x,:,:],cmap="gray", vmin=4000, vmax=45000)
# Trace histogram
counts, bins = exposure.histogram(Images)
a2[0][1].plot(bins, counts, '-')

a2[1][0].imshow(Med_3[i_x,:,:],cmap="gray", vmin=4000, vmax=45000)
# Trace histogram
counts, bins = exposure.histogram(Med_3)
a2[1][1].plot(bins, counts, '-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Pores + pure Polymer = Matrix
PP=(Images<=19*10**3).astype(int)#pores + their interfaces + pure polymer

# Pure pores
Pores=(Med_3<=11.7*10**3).astype(int) #pure pores before morphological operations #Histogram of pure phases is gaussian!
Pores_fill=(ndi.binary_fill_holes(Pores)).astype(int)
Pores_fill_open=(binary_opening(Pores_fill,ball(2))).astype(int)
Pores_fill_close=binary_closing(Pores_fill_open,ball(2)).astype(int)
Pores_fill_close=(ndi.binary_fill_holes(Pores_fill_close)).astype(int)

#Check pure pores modified by morphological operations
fig,(ax1, ax2,ax3)=  plt.subplots(1,3, figsize=(10, 5))
ax1.imshow(Pores_fill[100,:,:], cmap="gray")
ax2.imshow(Pores_fill_open[100,:,:], cmap="gray")
ax3.imshow(Pores_fill_close[100,:,:], cmap="gray")

Pure_Pores=np.logical_and(Pores_fill_close,PP).astype(int)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
#Show original image & pure pores
fig,(ax1, ax2)=  plt.subplots(1,2, figsize=(10, 5))
ax1.imshow(Images[100,:,:], cmap="gray")
ax2.imshow(Pure_Pores[100,:,:], cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Pore interfaces
Pores_Interface=binary_dilation(Pure_Pores,ball(4)).astype(int)-Pure_Pores # interfaces are in shpere of 4 voxels

PP_Interface=Pores_fill - np.logical_and(Pores_fill,Pores_fill_open).astype(int)
PP_Interfaces=PP_Interface - np.logical_and(PP_Interface,Pure_Pores).astype(int)
Pore_Matrix_Inter=binary_dilation(PP_Interfaces,ball(1)).astype(int) ## interfaces dilated 1 voxel
Pore_Matrix_Interf=Pore_Matrix_Inter-np.logical_and(Pore_Matrix_Inter,Pores_Interface).astype(int)-np.logical_and(Pore_Matrix_Inter,Pure_Pores).astype(int)

# All pore interfaces
Pores_Interfaces=Pores_Interface+Pore_Matrix_Interf
Pores_Interfaces=Pores_Interfaces-np.logical_and(Pores_Interfaces,Pure_Pores).astype(int)

# Pore interfaces in PP & the rest
PPP_Interfaces=np.logical_and(Pores_Interfaces,PP).astype(int)
Pores_BPP_Interface=Pores_Interfaces-PPP_Interfaces

In [9]:
#Powders + All particles
Matrix=PP-Pure_Pores-PPP_Interfaces
plt.figure()
plt.imshow(Matrix[100,:,:], cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Pure Polymes-Matrix
Pure_Matrix=binary_closing(binary_opening(Matrix, ball(2)), ball(2)).astype(int)
Pure_Matrix=Pure_Matrix-np.logical_and(Pure_Matrix,Pure_Pores).astype(int)-np.logical_and(Pure_Matrix,Pores_Interfaces).astype(int)

In [11]:
#Show original image & pure polymer
fig,(ax1, ax2)=  plt.subplots(1,2, figsize=(10, 5))
ax1.imshow(Images[100,:,:], cmap="gray")
ax2.imshow(Pure_Matrix[100,:,:], cmap="gray")

#Histogram: Pure pores/powders
fig,(ax1, ax2)=  plt.subplots(1,2, figsize=(10, 5))
counts, bins = exposure.histogram(Images*img_as_uint(Pure_Pores))
ax1.plot(bins, counts, '-')
counts, bins = exposure.histogram(Images*img_as_uint(Pure_Matrix))
ax2.plot(bins, counts, '-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\thi-xiu.le\AppData\Local\Continuum\miniconda3\lib\site-packages\skimage\util\dtype.py:454: UserWarning: Downcasting int32 to uint16 without scaling because max value 1 fits in uint16
  return convert(image, np.uint16, force_copy)
C:\Users\thi-xiu.le\AppData\Local\Continuum\miniconda3\lib\site-packages\skimage\util\dtype.py:454: UserWarning: Downcasting int32 to uint16 without scaling because max value 1 fits in uint16
  return convert(image, np.uint16, force_copy)


In [12]:
print(bins.shape)
print(counts.shape)

(256,)
(256,)


In [14]:
# fit a gaussian function to histogram of pure phases
from numpy import arange
import math
from scipy.optimize import curve_fit
 
# define the true objective function
def objective(x, A, m, sig):
	return A/(np.sqrt(2*math.pi)*sig) * np.exp(-(x-m)**2 / (2*sig**2)) 
# load the dataset
# choose the input and output variables
x, y = bins[1:], counts[1:]
# curve fit
A_0 = np.max(y)
m_0 = np.max(x * y/A_0)
sig_0 = 500#sum(y * (x - m)**2)
print('A_ini = %.5f, a_ini = %.5f, b_ini =  %.5f' % (A_0, m_0, sig_0))
popt, _ = curve_fit(objective, x, y, p0 = [A_0, m_0, sig_0])
# summarize the parameter values
A, m, sig = popt
print('A = %.5f, a = %.5f, b =  %.5f' % (A, m, sig))
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 50)
# calculate the output for the range
y_line = objective(x_line, *popt)
# create a line plot for the mapping function
plt.figure()
# plot input vs output
plt.scatter(x, y)
plt.plot(x_line, y_line, '--', color='red')
plt.show()

A_ini = 544110.00000, a_ini = 15477.31465, b_ini =  500.00000
A = 3598514538.13688, a = 15180.63966, b =  2617.49093


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
#Pore-Polymer/Particle Interfaces
Pore_Matrix_Interfaces=np.logical_and(binary_dilation(Pure_Matrix,ball(5)),binary_dilation(Pure_Pores,ball(5))).astype(int)
Pore_Matrix_Interfaces=Pore_Matrix_Interfaces - np.logical_and(Pore_Matrix_Interfaces,Pure_Matrix).astype(int) - np.logical_and(Pore_Matrix_Interfaces,Pure_Pores).astype(int)
Pore_Matrix_Interfaces=Pore_Matrix_Interf + Pore_Matrix_Interfaces - np.logical_and(Pore_Matrix_Interf,Pore_Matrix_Interfaces).astype(int) 
Pore_Matrix_Interfaces_Final=np.logical_and(Pores_Interface,Pore_Matrix_Interfaces).astype(int)
Pore_Particle_Interfaces=Pores_Interface - Pore_Matrix_Interfaces_Final

In [16]:
fig,(ax1, ax2)=  plt.subplots(1,2, figsize=(10, 5))
ax1.imshow(Pore_Matrix_Interfaces_Final[100,:,:], cmap="gray")
ax2.imshow(Pore_Particle_Interfaces[100,:,:], cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# Particles + Interfaces_Particles
Particles_Pre=(Med_3>27.5*10**3).astype(int)
Particles_Pre=Particles_Pre-np.logical_and(Particles_Pre,Pore_Particle_Interfaces).astype(int)
Particle=binary_opening(Particles_Pre,ball(1)).astype(int)
Particle=binary_closing(Particle,ball(1)).astype(int)
Particle=(ndi.binary_fill_holes(Particle)).astype(int)
Particles=Particle-np.logical_and(Particle,Pure_Pores).astype(int)-np.logical_and(Particle,Pure_Matrix).astype(int)-np.logical_and(Particle,Pore_Matrix_Interfaces_Final).astype(int)-np.logical_and(Particle,Pore_Particle_Interfaces).astype(int)

Interfaces_Particles=(Images>=0).astype(int)-(Pure_Pores+Pure_Matrix+Pore_Matrix_Interfaces_Final+Pore_Particle_Interfaces+Particles) #Matrix-Particle interfaces

In [18]:
#Show histogram of particles
plt.figure()
counts, bins = exposure.histogram(Images*img_as_uint(Particles))
plt.plot(bins, counts, '-')

#Show original image & pure particles
fig,(ax1, ax2)=  plt.subplots(1,2, figsize=(10, 5))
ax1.imshow(Images[100,:,:], cmap="gray")
ax2.imshow(Particles[100,:,:], cmap="gray")

#Show original image & polyemer_particle interfaces
fig,(ax1, ax2)=  plt.subplots(1,2, figsize=(10, 5))
ax1.imshow(Images[100,:,:], cmap="gray")
ax2.imshow(Interfaces_Particles[100,:,:], cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\thi-xiu.le\AppData\Local\Continuum\miniconda3\lib\site-packages\skimage\util\dtype.py:454: UserWarning: Downcasting int32 to uint16 without scaling because max value 1 fits in uint16
  return convert(image, np.uint16, force_copy)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
#Show images & pure phases
fig,a2=  plt.subplots(2,2, figsize=(10, 10))

a2[0][0].imshow(Images[i_x,:,:],cmap="gray")
a2[0][1].imshow(Particles[i_x,:,:],cmap="gray")
a2[1][0].imshow(Pure_Pores[i_x,:,:],cmap="gray")
a2[1][1].imshow(Pure_Matrix[i_x,:,:],cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
Check=Pure_Pores+Pure_Matrix+Particles+Pore_Matrix_Interfaces_Final+Pore_Particle_Interfaces+Interfaces_Particles
print(np.min(Check))
print(np.max(Check))

1
1


In [21]:
V_Pure_Pores=np.count_nonzero(Pure_Pores)
V_Pure_Matrix=np.count_nonzero(Pure_Matrix)
V_Particles=np.count_nonzero(Particles)

V_Pore_Matrix=np.count_nonzero(Pore_Matrix_Interfaces_Final)
V_Pore_Particle=np.count_nonzero(Pore_Particle_Interfaces)
V_Matrix_Particle=np.count_nonzero(Interfaces_Particles)

print(V_Pure_Pores+V_Pure_Matrix+V_Particles+V_Pore_Matrix+V_Pore_Particle+V_Matrix_Particle)

50000000


In [22]:
M_Pores=np.sum(Images*img_as_uint(Pure_Pores))/V_Pure_Pores
M_Matrix=np.sum(Images*img_as_uint(Pure_Matrix))/V_Pure_Matrix
M_Particles=np.sum(Images*img_as_uint(Particles))/V_Particles

Std_Pore=np.sqrt(np.sum((Images*img_as_uint(Pure_Pores))**2)/V_Pure_Pores-M_Pores**2)
Std_Matrix=np.sqrt(np.sum((Images*img_as_uint(Pure_Matrix))**2)/V_Pure_Matrix-M_Matrix**2)
Std_Particle=np.sqrt(np.sum((Images*img_as_uint(Particles))**2)/V_Particles-M_Particles**2)

M_Pore_Matrix=np.sum(Images*img_as_uint(Pore_Matrix_Interfaces_Final))/V_Pore_Matrix
M_Pore_Particle=np.sum(Images*img_as_uint(Pore_Particle_Interfaces))/V_Pore_Particle
M_Matrix_Particle=np.sum(Images*img_as_uint(Interfaces_Particles))/V_Matrix_Particle

print(M_Pores,M_Matrix,M_Particles,M_Pore_Matrix,M_Pore_Particle,M_Matrix_Particle)
print(Std_Pore,Std_Matrix,Std_Particle)

C:\Users\thi-xiu.le\AppData\Local\Continuum\miniconda3\lib\site-packages\skimage\util\dtype.py:454: UserWarning: Downcasting int32 to uint16 without scaling because max value 1 fits in uint16
  return convert(image, np.uint16, force_copy)


9072.77154722226 15158.003055622752 35486.73238681446 15828.15366267883 15890.097110465329 20298.899206793194
2773.7437362149203 2624.76525452136 6045.899569932203


In [23]:
V_Pores=V_Pure_Pores + (M_Matrix-M_Pore_Matrix)/(M_Matrix-M_Pores)*V_Pore_Matrix + (M_Particles-M_Pore_Particle)/(M_Particles-M_Pores)*V_Pore_Particle 
V_Matrix=V_Pure_Matrix + (M_Pore_Matrix-M_Pores)/(M_Matrix-M_Pores)*V_Pore_Matrix + (M_Particles-M_Matrix_Particle)/(M_Particles-M_Matrix)*V_Matrix_Particle 
V_Particles=V_Particles + (M_Pore_Particle-M_Pores)/(M_Particles-M_Pores)*V_Pore_Particle + (M_Matrix_Particle-M_Matrix)/(M_Particles-M_Matrix)*V_Matrix_Particle 

V_all=(V_Pores+V_Matrix+V_Particles)
print(V_all)

50000000.0


In [24]:
Porosity=V_Pores/V_all*100
Fiber=(V_Particles)/V_all*100
print(Porosity,Fiber)

2.506995601569607 14.081436239037014


In [25]:
t=100
Color_Image = 0*Pure_Pores[t,:,:] + 50*Pore_Matrix_Interfaces_Final[t,:,:] + 100*Pore_Particle_Interfaces[t,:,:]  + 150*Pure_Matrix[t,:,:] + 200*Interfaces_Particles[t,:,:]+ 255*Particles[t,:,:] 

fig,(ax1, ax2)=  plt.subplots(1,2, figsize=(10, 5))
ax1.imshow(Images[t,:,:], cmap="gray")
ax2.imshow(Color_Image, cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
import scipy.io

# Emplacement des fichiers
dir_data = pathlib.Path('Test3_1751_1950')
path = dir_data 

for t in range (0,200):
    Color_Image =  0*Pure_Pores[t,:,:] + 50*Pore_Matrix_Interfaces_Final[t,:,:] + 100*Pore_Particle_Interfaces[t,:,:]  + 150*Pure_Matrix[t,:,:] + 200*Interfaces_Particles[t,:,:]+ 255*Particles[t,:,:] 
    io.imsave(str(path/'Color_Images'/'%05d.tif')%(t),img_as_ubyte(Color_Image)) 